# Earth / Moon vs. Earth / Mars Barcodes

### Setup Base

In [1]:
from os_utilities import *
from orb_builder import *
import time_utilities as tu

orb_filenames = [] # used to keep track of what files to feed into soap


prefix = "starlink" # earth / moon / mars
n = 30 # number of simulations to generate
k = 15 # number of satellites to sample from starlink

folder_name = "sim-2023-03-23-rnd"
folder_name = "starlink-{}-sat-single".format(k)

folder = "./outputs/" + folder_name
make_folder(folder)

starlink = build_satellites_csv("starlink")
moon_sats = get_moon_platforms()
mars_sats = get_mars_platforms()
ground = [
    create_ground_object_dictionary("Ground-Albany", 42.685012663456163, -73.82479012295363, 0),
    create_ground_object_dictionary("Ground-Sydney", -33.868888888888889, 151.20939697339508, 0)
]
norad = build_satellites_from_tle("./sources/tdrss.tle")

import datetime
import random

year = 2022
months = 24 # 8 years
months = 1 # 8 years
month_days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

dates = []
for j in range(months):
    
    if j > 0 and j % 12 == 0:
        year += 1
    month_index = j % 12
    day = random.randint(1, month_days[month_index])
    day = 1
    # date = datetime.datetime(year, month_index + 1, day)
    date = tu.get_random_date()
    dates.append(date)
# print(len(dates))
# print(dates)
# dates = []


In [2]:
for i in range(n):
    for d in dates:
        name = prefix + "_" + str(k) + "_sats_" + d.strftime("%Y-%m") + "_" + str(i) # "%Y-%m-%d"
        name = prefix + "_" + str(k) + "_sats_" + str(i) # "%Y-%m-%d"

        subset = sample_sats(starlink, k)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset # + moon_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Earth / Moon System

In [2]:
for i in range(n):
    for d in dates:

        name = prefix + "_moon_" + d.strftime("%Y-%m") + "_" + str(i)

        subset = sample_sats(starlink, k)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset + moon_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Earth / Mars System

In [3]:
for i in range(n):
    for d in dates:

        name = prefix + "_mars_" + d.strftime("%Y-%m") + "_" + str(i)

        subset = sample_sats(starlink, k + 1)

        # track which satellites were chosen for subsequent simulations
        subset_names = []
        for satellite in subset:
            subset_names.append(satellite["object_name"])

        satellites = subset + mars_sats # + ground

        orb_filenames.append(name + ".orb")

        save_to_outputs_file(
            generate_orb(satellites, name, d),
            folder_name + "/" + name, "orb")

## Run Soap

In [3]:
# folder = "outputs/starlink-6"
# orb_files = get_ext_files(folder, "orb")
# orb_filenames = []
# for orb_file in orb_files:
#     orb_filenames.append(orb_file.split("/")[-1])
# print(orb_filenames)
bash = starlink_bash = "#!/bin/bash" + "\n"

for filename in orb_filenames:
    bash += f"echo 'open -n -j -W {filename}'\n"
    bash += f"open -n -j -W {filename}\n"
save_to_outputs_file(
        bash,
        folder_name + "/run",
        "sh")

# hack to get around soap bug
#   soap doesn't close after reports are completed when in batch mode in macos
#   but it should
osascript_unfocus = [
    'osascript',
    '-e',
    'tell application "System Events"',
    '-e',
    'set visible of application process "SOAP" to false',
    '-e',
    'end tell'
]
print(" ".join(osascript_unfocus))

osascript_focus = [
    'osascript',
    '-e',
    'tell application "System Events"',
    '-e',
    'tell process "SOAP"',
    '-e',
    'set frontmost to true',
    '-e',
    'end tell',
    '-e',
    'end tell'
]
print(" ".join(osascript_focus))

osascript -e tell application "System Events" -e set visible of application process "SOAP" to false -e end tell
osascript -e tell application "System Events" -e tell process "SOAP" -e set frontmost to true -e end tell -e end tell


In [4]:
#!/usr/bin/python

import Quartz
import time
from Quartz import CGWindowListCopyWindowInfo, kCGWindowListExcludeDesktopElements, kCGNullWindowID
from Foundation import NSSet, NSMutableSet

wl1 = CGWindowListCopyWindowInfo(kCGWindowListExcludeDesktopElements, kCGNullWindowID)
wl = Quartz.CGWindowListCopyWindowInfo( Quartz.kCGWindowListOptionAll, Quartz.kCGNullWindowID)
wl = sorted(wl, key=lambda k: k.valueForKey_('kCGWindowOwnerPID'))

def soapWindowList(wl):
    for v in wl:
        if v.valueForKey_('kCGWindowOwnerName') == "SOAP":
            print ( 
            str(v.valueForKey_('kCGWindowOwnerPID') or '?').rjust(7) + 
                ' ' + str(v.valueForKey_('kCGWindowNumber') or '?').rjust(5) + 
                ' {' + ('' if v.valueForKey_('kCGWindowBounds') is None else ( 
                        str(int(v.valueForKey_('kCGWindowBounds').valueForKey_('X')))     + ',' + 
                        str(int(v.valueForKey_('kCGWindowBounds').valueForKey_('Y')))     + ',' + 
                        str(int(v.valueForKey_('kCGWindowBounds').valueForKey_('Width'))) + ',' + 
                        str(int(v.valueForKey_('kCGWindowBounds').valueForKey_('Height'))) 
                    ) ).ljust(21) + '}' + 
                '\t[' + ((v.valueForKey_('kCGWindowOwnerName') or '') + ']') + 
                ('' if v.valueForKey_('kCGWindowName') is None else (' ' + 
                v.valueForKey_('kCGWindowName') or '')) 
            )
# soapWindowList(wl)

commands = []
for filename in orb_filenames:
    # args = ["open", "-n", "-W", f"{folder}/{filename}"]
    args = ["open", "-n", "-j", "-W", f"{folder}/{filename}"]
    commands.append(args)

print(f"len(commands)={len(commands)}")

len(commands)=30


In [5]:
import subprocess
from itertools import islice

max_workers = 8  # no more than 2 concurrent processes
processes = (subprocess.Popen(cmd) for cmd in commands)
running_processes = list(islice(processes, max_workers))  # start new processes
while running_processes:
    for i, process in enumerate(running_processes):
        if process.poll() is not None:  # the process has finished
            running_processes[i] = next(processes, None)  # start new process
            if running_processes[i] is None: # no new processes
                del running_processes[i]
                break
        else:
            time.sleep(0.5)
            subprocess.run(osascript_unfocus)        
            subprocess.run(osascript_focus)

31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


In [4]:
import subprocess
import time

command = "wine"
soap_path = "/home/zeno/.wine/drive_c/SOAP/bin64/soap.exe"

command = "open"
soap_path = ""

for filename in orb_filenames:
    # args = ["open", "-n", "-W", f"{folder}/{filename}"]
    args = ["open", "-n", "-j", "-W", f"{folder}/{filename}"]
    print(" ".join(args))
    # orb_path = "" + folder + "/" + filename + ""
    # process = subprocess.Popen(args)
    # process = subprocess.Popen([command, orb_path], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
    process = subprocess.Popen(args)
    print(process.pid)
    while process.poll() is None:
        time.sleep(0.5)
        subprocess.run(osascript_unfocus)        
        subprocess.run(osascript_focus)
    # process.wait()

open -n -j -W ./outputs/starlink-100-sat-single/starlink_100_sats_0.orb
53489


31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


open -n -j -W ./outputs/starlink-100-sat-single/starlink_100_sats_1.orb
53542


31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


open -n -j -W ./outputs/starlink-100-sat-single/starlink_100_sats_2.orb
53589


...

31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


open -n -j -W ./outputs/starlink-100-sat-single/starlink_100_sats_28.orb
54860


31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


open -n -j -W ./outputs/starlink-100-sat-single/starlink_100_sats_29.orb
54909


31:84: execution error: System Events got an error: Can’t set application process "SOAP" to false. (-10006)
57:66: execution error: System Events got an error: Can’t set process "SOAP" to true. (-10006)


## Generate Barcodes

In [ ]:
from zz_persistence import *

diagrams = {}
diagrams_clique = {}

filepaths = get_csv_files(folder)
for filepath in filepaths:
    if not filepath.endswith("Contact Analysis.csv"):
        continue
    contact_plan = ca.contact_analysis_parser(filepath)
    graph = ca.construct_graph(contact_plan)
    
    weighted_simplex = ca.construct_weighted_simplex(graph)
    clique_complex = ca.construct_clique_complex(graph)
    
    zz, dgms, cells = calculate_zz_persistence(weighted_simplex)
    zz_clique, dgms_clique, cells_clique = calculate_zz_persistence(clique_complex)
    
    diagrams[filepath.split("/")[-1]] = dgms
    diagrams_clique[filepath.split("/")[-1]] = dgms_clique
    
# print(diagrams)
# print(diagrams_clique)

In [ ]:
dim = 1

## Compare Earth / Moon Simulations

In [ ]:
# Compare Earth / Moon
for i in range(n):
    for j in range(i + 1, n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_moon_" + str(j) + " Contact Analysis.csv"]
        
        diagram_a_clique = diagrams_clique["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_moon_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        

## Compare Earth / Mars Simulations

In [ ]:
# Compare Earth / Mars
for i in range(n):
    for j in range(i + 1, n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_mars_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_mars_" + str(j) + " Contact Analysis.csv"]

        diagram_a_clique = diagrams_clique["emm_mars_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_mars_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        

## Cross Compare Moon / Mars

In [ ]:
# Cross Compare Moon / Mars
for i in range(n):
    for j in range(n):
        print("{} : {}".format(i, j))

        diagram_a = diagrams["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b = diagrams["emm_mars_" + str(j) + " Contact Analysis.csv"]
        
        diagram_a_clique = diagrams_clique["emm_moon_" + str(i) + " Contact Analysis.csv"]
        diagram_b_clique = diagrams_clique["emm_mars_" + str(j) + " Contact Analysis.csv"]

        wdist = d.wasserstein_distance(diagram_a[dim], diagram_b[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams: {}".format(dim, wdist))

        bdist = d.bottleneck_distance(diagram_a[dim], diagram_b[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams: {}".format(dim, bdist))
        
        wdist_clique = d.wasserstein_distance(diagram_a_clique[dim], diagram_b_clique[dim], q=2)
        print("2-Wasserstein distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, wdist_clique))

        bdist_clique = d.bottleneck_distance(diagram_a_clique[dim], diagram_b_clique[dim])
        print("Bottleneck distance between {}-dimensional persistence diagrams (Clique): {}".format(dim, bdist_clique))
        
        